In [3]:
#!/usr/bin/python
import os
from  osgeo import ogr, osr
import pickle
from shapely.geometry import LinearRing,Point,mapping
import numpy as np
#np.set_printoptions(precision=12)
np.set_printoptions(suppress=False)
INF = 999

def eqsegment(seg1p1,seg1p2,seg2p1,seg2p2,labeling=False): #tuples of points
    if not labeling:
        s0=sorted([seg1p1,seg1p2])
        s1=sorted([seg2p1,seg2p2])
        if s0[0][0] == s1[0][0] and s0[0][1] == s1[0][1] and s0[1][0] == s1[1][0] and s0[1][1] == s1[1][1]:
            return True
        return False
    
def GetSegmentsAsTuple(rings,ringids):
    L= []
    if type(rings) != list:
        rings=[rings]
    if type(ringids) !=list:
        ringids=[ringids]
    if len(rings) != len(ringids):
        raise ValueError("Size of rings and their ids did not match.")
        
    L = np.zeros((1,6)) #dummy variable for [rid,x1,y1,x2,y2,isswp?]
    for ringid in xrange(len(rings)):
        arr=None
        ring=rings[ringid]
        if not ring.is_ccw:
            raise ValueError("Polygon ring at index %r is not in anti-clockwise order."%ringid)
            
        arr = np.asarray(list(ring.coords)) #x1,y1,z1
        if ring.has_z:
            arr= arr[:,0:2] #exclude z.
        arr = np.hstack((arr[0:-1,:],arr[1:,:])) #shifting x1,y1,x2,y2
        rids = np.asarray([ringids[ringid]]*arr.shape[0]).reshape((arr.shape[0],1))
        isswapped = np.zeros(rids.shape)
        arr = np.hstack((rids,arr,isswapped)) #add 'inf' as last column. also add isvisited after isswp?
        #arr --> [rid,x1,y1,x2,y2,iswp?,rid-left]
        def swap_endpoints(c):
            if c[3] < c[1]:
                t1,t2 = c[1],c[2]
                c[1]= c[3];c[2] =c[4];c[3] = t1;c[4] = t2; c[5] = 1
            return c
        arr = np.apply_along_axis(swap_endpoints,1,arr) #arr --> [rid,x1,y1,x2,y2,iswp?]
        L = np.vstack((L,arr))
    #end-for
    m,n = L.shape
    inf = np.asarray([INF]*(m-1)).reshape(m-1,1) #without dummy first row; right-labels for each rid.
    L = np.hstack((L[1:,:],L[1:,0:1],inf)) #arr--> [rid,x1,y1,x2,y2,iswp?,L-labels,R-labels] #left-labels for each rid.
    print("Constructed segment tuples of size:"), L.shape
    return L

def GetTopBottomLabelSegs(L):
    print("Top/Bottom labeling started; got segment-tuples of size "), L.shape, type(L)
    cnt=0
    L = L[L[:,1].argsort()] #sort by x1.;#L= sorted(L, key=lambda t: t[1]) 
    #append isvisited? column
    isvisited = np.zeros((len(L),1))
    L = np.hstack((L,isvisited)) #L's columns--> [rid,x1,y1,x2,y2,iswp?,rid-left,rid-right,isvisited?]
    for ti in range(len(L)): #each tuple-index
        t = L[ti]
        polyid,s1x1,s1y1,s1x2,s1y2,s1isswapped = t[0:6]
        #check is swapped.
        if s1isswapped:x1,y1,x2,y2 = s1x2,s1y2,s1x1,s1y1 #get original arrangement of points for the seg.
        else:x1,y1,x2,y2 = s1x1,s1y1,s1x2,s1y2
        if t[-1] == False: #not visited
            for nti in range(ti+1,len(L)):#new tuple index
                newt = L[nti]
                polyid_p,s2x1,s2y1,s2x2,s2y2,s2isswapped = newt[0:6]
                if s1x1 < s2x1: #no matching segment found
                    break
                if eqsegment((s1x1,s1y1),(s1x2,s1y2),(s2x1,s2y1),(s2x2,s2y2)):
                    cnt+=1
                    #visited !!
                    t[-1]=True
                    newt[-1]=True #visited
                    #now do casw-wise.
                    if x1 < x2: #not swapped
                        t[6] = polyid #top
                        t[7] = polyid_p #bot
                        newt[6] =  polyid#same code.
                        newt[7] = polyid_p
                        break
                    elif x1 > x2:
                        t[6] = polyid_p#top
                        t[7] = polyid#bot
                        newt[6] = polyid_p
                        newt[7] = polyid
                        break
                    elif x1 == x2:
                        if y1 < y2: #vup
                            t[6] = polyid #top
                            t[7] = polyid_p #bot
                            newt[6] = polyid_p
                            newt[7] = polyid  
                            break
                        elif y1 > y2: #vdown
                            t[6] = polyid_p #top
                            t[7] = polyid #bot
                            newt[6] = polyid_p  
                            newt[7] = polyid   
                            break  #exit form current loop.
            #end-for
            if t[-1] == False: #this segment is no shared by any other rings.
                t[-1]=True #now visti
                if x1 < x2:
                    t[6] = polyid #top
                    t[7] = INF #bot
                elif x1 > x2:
                    t[6] = INF #top
                    t[7] = polyid #bot
                elif x1 == x2:
                    if y1 < y2: #vup
                        t[6] = polyid #top
                        t[7] = INF #bot
                    elif y1 > y2: #vdown
                        t[6] = INF #top
                        t[7] = polyid #bot
            #end-if
        #end-if
    L = L[0:,0:-1]#exclude isvisited col.
    sL = L[L[:,0].argsort()] #sort by state-id
    return sL

def GetOutfeature(outLayer, outLayerDefn,geom,inFeature): #copy infeature to outLayer, and sets geom into feature.
    outFeature = ogr.Feature(outLayerDefn)
    outFeature.SetGeometry(geom)
    for i in range(0, outLayerDefn.GetFieldCount()):
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
    return outFeature

def createOutLayer(inlayer,outputShapefile, outlayername,outprojref,outgeomtype=ogr.wkbMultiPolygon):
    #create a data-set.
    outLayer=outLayerDefn=None
    
    if os.path.exists(outputShapefile):
        driver.DeleteDataSource(outputShapefile)
    outDataSet = driver.CreateDataSource(outputShapefile)

    # create output layer, copy fields from inlayer fielddefn
    outLayer = outDataSet.CreateLayer(outlayername,outprojref,geom_type=outgeomtype)

    inLayerDefn = inlayer.GetLayerDefn()
    for i in range(0, inLayerDefn.GetFieldCount()):
        fieldDefn = inLayerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)

    outLayerDefn = outLayer.GetLayerDefn()
    return outDataSet,outLayer, outLayerDefn

'''Testing Polygon Segment Labeling'''
'''Test Code '''
ring1=[(5.0, 5.0, 0.0), (6.0, 4.0, 0.0), (8.0, 4.0, 0.0), (8.0, 2.0, 0.0), (10.0, 4.0, 0.0), (11.0, 3.0, 0.0), (12.0, 5.0, 0.0), (10.0, 6.0, 0.0),
 (9.0, 5.0, 0.0), (6.0, 6.0, 0.0), (5.0, 5.0, 0.0)]
ring2=[(14,1),(17,2),(17,4),(15,4),(14,7),(14,8),(9,8),(9,5),(10,6),(12,5),(11,3),(13,3),(14,1)]
shring1 = LinearRing(ring1)
shring2 = LinearRing(ring2)
rings=[shring1,shring2]
L= GetSegmentsAsTuple(rings,[0,1]) #list of segments
print
Lp= GetTopBottomLabelSegs(L).tolist()
Lp= sorted(Lp, key=lambda t: t[0]) 
for tup in Lp:
    print tup
print("Testing Completed.")
print
L=Lp=ring2=shring1=shring2=rings=None
##===========================END TESTING==============================##


def label_segments_inmap(srcshpfile):
    
    driver = ogr.GetDriverByName('ESRI Shapefile')
    shp = driver.Open(srcshpfile)
    inlayer = shp.GetLayer()
    #outDataSet,outLayer, outLayerDefn = createOutLayer(inlayer,None,"segment-labeled",inlayer.GetSpatialRef(),outgeomtype=ogr.wkbMultiPolygon)
    import time
    #read the features one by one.
    inFeature = inlayer.GetNextFeature()
    rings=[]
    ringids=[]
    sttime=time.time()
    while inFeature:
        geom= inFeature.GetGeometryRef()
        if geom.GetGeometryName() == 'POLYGON':
            if geom.GetGeometryCount()>=1:
                for i in xrange(geom.GetGeometryCount()):
                    ring= geom.GetGeometryRef(i)
                    shapelyRing= LinearRing(ring.GetPoints())
                    if not shapelyRing.is_ccw:
                        shapelyRing= LinearRing(reversed(ring.GetPoints()))
                    rings+=[shapelyRing]
                    try:
                        print("ringid"),int(inFeature.GetField(0))
                        ringids+=[int(inFeature.GetField(0))]#ringids+=['p'+str(inFeature.GetField(0))+'r'+str(i)]
                    except:
                        raise ValueError("Feature ids must be integer.")
        inFeature = inlayer.GetNextFeature()
    print time.time()-sttime
    #end-while
    
    #start labeling segments
    sttime=time.time()
    segment_tups= GetSegmentsAsTuple(rings[0:],ringids[0:])#tuples.
    print("time getSegmenstsAsTuple"),time.time()-sttime, segment_tups.shape,len(rings),len(ringids)
    print("counts of unique columns"),len(np.unique(segment_tups[0:,0])),len(np.unique(segment_tups[0:,1]))
    inFeature=rings=ringids=inLayer=driver=shp=None
    
    ###
    sttime=time.time()
    labeled_stups= GetTopBottomLabelSegs(segment_tups)#labeled tuples.
    print("time GetTopBottomLabelsSegs"),time.time()-sttime, labeled_stups.shape
    return labeled_stups


home=srcshpfile=outputShapefile=labeled_segments_array=None
home =r'/home/naresh-1/workspace/machinelrn/data/gis/'
srcshpfile = home + r'out/prj_states.shp'
outputfile=home+r"out/prj.states.lbl.dbf" #will contain labels for each segments in the map
labeled_segments_array = label_segments_inmap(srcshpfile)

Constructed segment tuples of size: (22, 8)

Top/Bottom labeling started; got segment-tuples of size  (22, 8) <type 'numpy.ndarray'>
[0.0, 5.0, 5.0, 6.0, 4.0, 0.0, 0.0, 999.0]
[0.0, 11.0, 3.0, 12.0, 5.0, 0.0, 0.0, 1.0]
[0.0, 10.0, 6.0, 12.0, 5.0, 1.0, 1.0, 0.0]
[0.0, 9.0, 5.0, 10.0, 6.0, 1.0, 1.0, 0.0]
[0.0, 8.0, 2.0, 10.0, 4.0, 0.0, 0.0, 999.0]
[0.0, 10.0, 4.0, 11.0, 3.0, 0.0, 0.0, 999.0]
[0.0, 6.0, 6.0, 9.0, 5.0, 1.0, 999.0, 0.0]
[0.0, 6.0, 4.0, 8.0, 4.0, 0.0, 0.0, 999.0]
[0.0, 5.0, 5.0, 6.0, 6.0, 1.0, 999.0, 0.0]
[0.0, 8.0, 4.0, 8.0, 2.0, 0.0, 999.0, 0.0]
[1.0, 9.0, 5.0, 10.0, 6.0, 0.0, 1.0, 0.0]
[1.0, 9.0, 8.0, 9.0, 5.0, 0.0, 999.0, 1.0]
[1.0, 9.0, 8.0, 14.0, 8.0, 1.0, 999.0, 1.0]
[1.0, 15.0, 4.0, 17.0, 4.0, 1.0, 999.0, 1.0]
[1.0, 10.0, 6.0, 12.0, 5.0, 0.0, 1.0, 0.0]
[1.0, 11.0, 3.0, 13.0, 3.0, 0.0, 1.0, 999.0]
[1.0, 11.0, 3.0, 12.0, 5.0, 1.0, 0.0, 1.0]
[1.0, 13.0, 3.0, 14.0, 1.0, 0.0, 1.0, 999.0]
[1.0, 14.0, 7.0, 15.0, 4.0, 1.0, 999.0, 1.0]
[1.0, 14.0, 7.0, 14.0, 8.0, 0.0, 1.0, 99

In [95]:
import time
def save_labeled_segments(labeled_segments_array, home,outputfile):
    '''numpy array'''
    #Save output
    import numpy as np
    import pickle
    from dbfpy import dbf
    #load state-code file as pickle.
    if len(labeled_segments_array) == 0:
        raise ValueError("Cannot write to a dbf file. Input array empty. size."),labeled_segments_array.shape
        return save
    SC = pickle.load(open(home+'state_dic.p','rb'))
    db = dbf.Dbf(outputfile, new=True)
    
    db.addField(
        ("rid", "N",5),
        ("x1", "F",20,12),
        ("y1", "F", 20,12),
        ("x2", "F", 20,12),
        ("y2", "F", 20,12),
        ("isswp","N",1),
        ("ridtop","N",5),
        ("ridbot","N",5),
        ('nametop',"C",50),
        ('namebot',"C",50)
    )
    
    st= time.time()
    for item in labeled_segments_array:
        rid,x1,y1,x2,y2,isswp,ridtop,ridbot=tuple(item)
        rec = db.newRecord()
        rec['rid']=rid
        rec['x1']=x1
        rec['y1']=y1
        rec['x2']=x2
        rec['y2']=y2
        rec['isswp']=isswp
        rec['ridtop'] = ridtop
        rec['ridbot'] = ridbot
        try:rec['nametop'] = SC[ridtop]
        except:rec['nametop'] = ridtop #if not exists.
        #bottom name.
        try:rec['namebot'] = SC[ridbot]
        except:rec['namebot'] = ridbot
        rec.store()
    
    db.close()
    
    print("Time required to create .dbf"), time.time()-st
    print  "columns: polyid,x1,y1,x2,y2,isswp,lbltop,lblbot are written"
    print(".dbf file saved in "),outputfile
    return SC, outputfile

len(labeled_segments_array[0:10,:])

10